Elizabeth Avery Frankenberg; Machine Learning Homework 3; SVM Classification; February 10 2019

In [119]:
#the necessary import statements
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import KFold, cross_validate
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [108]:
#importing the data and separating it into labels and features
data = pd.read_csv("hw3_training_data.csv")

data_labels = data['Label']
data_features = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]
data_subset = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_6']]

data_features.head()

#print data_labels

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6
0,9070.02,20594.1000,-3429.72,94,163,1162
1,3747.51,1137.0000,-872.00,10,52,101
2,2821.21,50.5575,-589.00,6,33,5
3,10600.90,28167.8000,-2640.00,30,194,1521
4,13579.40,98062.5000,-5354.46,162,281,4061


First, I created a train, test method which runs K fold cross validation on the SVC or NuSVC and returns the accuracy obtained. 

In [118]:
#this method takes a subset and trains and tests it, returning its score
def train_test(data_subset):
    kf = KFold(n_splits = 10)
    avg_accuracy = []

    for train, test in kf.split(data_features):
        labels_train, data_train = data_labels.iloc[train], data_subset[train]
        labels_test, data_test = data_labels.iloc[test], data_subset[test]
        clf = svm.NuSVC( cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovr', degree=3, gamma=.000001, kernel='rbf',
        max_iter=-1, probability=False, random_state=None, shrinking=True,
        tol=0.001, verbose=False)
        #norm_data = preprocessing.normalize(data_train, axis=1)
        #clf = make_pipeline(StandardScaler(), svm.SVC())
        clf.fit(data_train, labels_train) 
        avg_accuracy.append(clf.score(data_test, labels_test))
    
    avg = 0
    for i in range (0, 10):
        avg += avg_accuracy[i]
    print avg/10
    return clf

To try and improve my parameters I created svc param selection which conducts GridSearchCV on a specified parameter matrix of Cs, gammas, and alternating kernels. I used this and the subsequent cell to determine the best parameters to use in my SVC. I ran this for both NuSVC and SVC--however, when I ran it for NuSVC I took out the parameter C as this is not included in the model. I also modified the kernel type, to be linear, sigmoid or polynomial. As these had long run times, I ran them each seperately and then compared the best linear parameters to best sigmoid to best polynomial to best rbf. I found that the rbf kernel provided the best results.

In [124]:
#this method does parameter selection using grid search
def svc_param_selection(features, labels, nfolds):
    Cs = [0.001, .01, .1, 1, 10, 100, 1000]
    gammas = [0.00000001, 0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 1]
    kernels = ['rbf'] #can switch this to 'linear', 'sigmoid', or 'polynomial' also, ran these all separately for runtime
    params = {'C': Cs,'gamma' : gammas, 'kernel': kernels}
    grid_search = GridSearchCV(svm.SVC(), params, cv=nfolds)
    grid_search.fit(features, labels)
    grid_search.best_params_
    return grid_search.best_params_

In [125]:
best_params = svc_param_selection(data_features, data_labels, nfolds=10)
print best_params

{'kernel': 'rbf', 'gamma': 1e-06}


The best parameters for NuSVC were Kernel = rbf and Gamma = 1*10^-6. The best parameters for SVC were Kernel = rbf, C=1, Gamma = 1*10^-6. 

This method then goes through and tries a subset of features. I used this along with some manual attemps to find the best set of subsets possible. 

In [88]:
def try_all_features(data_features):
    best_score= 0
    features = None
    score_dict = dict()
    
    data_features = data_features.abs()
    
    for i in range(0, 6):
        for j in range(0, 6):
            
            if (j < i):
                data_new = np.vstack(np.array(data_features.iloc[:,j:i]))
        
            elif (i == j):
                data_new = np.vstack(np.array(data_features.iloc[:,i]))
        
            else:
                data_new = np.vstack(np.array(data_features.iloc[:,i:j]))
           
            split_avg = train_test(data_new)
            if (split_avg > best_score):
                best_score = split_avg
                features = data_new
                print best_score
    print ("Best score:", best_score)
    print ("Features:", features)

In [89]:
try_all_features(data_features)

0.59375
0.7237499999999999
0.7582142857142857
0.7598214285714285
('Best score:', 0.7598214285714285)
('Features:', array([[9.07002e+03, 2.05941e+04, 3.42972e+03, 9.40000e+01],
       [3.74751e+03, 1.13700e+03, 8.72000e+02, 1.00000e+01],
       [2.82121e+03, 5.05575e+01, 5.89000e+02, 6.00000e+00],
       ...,
       [1.20230e+04, 7.34458e+04, 4.61000e+03, 3.00000e+01],
       [2.94640e+04, 7.31255e+05, 1.46770e+04, 1.72000e+02],
       [9.79249e+03, 5.15269e+04, 3.11000e+03, 2.80000e+01]]))


In [117]:
train_test(np.array(data_subset))

0.7657142857142856

SVC:

Feature 1, 2, 3, 4
0.7594642857142857

Feature 1, 2, 3, 6
0.7608928571428571

Feature 1, 2, 3, 4, 6
0.7619642857142856

NuSVC:
Feature 1, 2, 3, 4, 6
0.7657142857142856

In [126]:
train_test(np.array(data_features))

0.7655357142857142


NuSVC(cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma=1e-06, kernel='rbf',
   max_iter=-1, nu=0.5, probability=False, random_state=None,
   shrinking=True, tol=0.001, verbose=False)

In [130]:
#this method trains the model on a given file
def train(filename):
    
    data = pd.read_csv(filename)
    data_labels = data['Label']
    data_features = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]
    data_subset = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_6']]
    return train_test(np.array(data_subset))

In [133]:
#this method does the prediction step and puts these values into a new file
def predict_file(C, feature_test_filename, predict_test_filename):
    test_data = pd.read_csv(feature_test_filename)
    with open(predict_test_filename, 'w') as writeFile:
        writer = csv.writer(writeFile) 
        data_features = test_data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_6']]
        writer.writerow(['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_6', 'Label'])
        X = np.array(data_features)
        line = 0
        while (line < 2400):
            row = X[line]
            prediction = predict(C, row)
            row = np.append(row, prediction)
            writer.writerow(row)
            line += 1
        

In [135]:
def predict(C, row):
    row = row.reshape(1, -1)
    prediction = C.predict(row)
    return prediction

These two methods train the model clf using the parameters and type of SVC that I found to be the most accurate. They then predict the labels for the rows in the test file, subsequently outputting them to a predictions.csv. 

In [137]:
clf = train("hw3_training_data.csv")
predict_file(clf, "hw3_test_data.csv", "hw3_predictions.csv")

0.7657142857142856
